In [7]:
#Import modules
import pandas as pd
import requests
from collections import defaultdict



import yfinance as yf
from datetime import datetime



In [8]:
#Extract updated list of SP500 tickers
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

user_agents = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

response = requests.get(url_sp500, headers=user_agents, timeout=30)
response.raise_for_status()

tables = pd.read_html(response.text)
df_sp500 = tables[0]

sector_to_tickers = defaultdict(list)

for symbol, sector in zip(df_sp500["Symbol"], df_sp500["GICS Sector"]):
    sector_to_tickers[sector].append(symbol)

dict_sp500 = dict(sector_to_tickers)

#Copy dict_sp500 to csv
rows = [
    {"Sector": sector, "Ticker Symbol": ticker}
    for sector, tickers in dict_sp500.items()
    for ticker in tickers
]

df_sp500_output = pd.DataFrame(rows)

df_sp500_output.to_csv("sp500_tickers.csv", index=False)

C:\Users\billg\AppData\Local\Temp\ipykernel_21108\1252989007.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [9]:

# -----------------------------
# Assumes df_sp500_output already exists
# Columns:
# Sector | Ticker Symbol | Last_Updated
# -----------------------------

# 1️⃣ Extract ticker list
tickers = df_sp500_output["Ticker Symbol"].dropna().unique().tolist()

print(f"Number of tickers: {len(tickers)}")

# -----------------------------
# 2️⃣ Download 10 years of daily prices
# -----------------------------
df_prices = yf.download(
    tickers=tickers,
    period="10y",
    interval="1d",
    group_by="ticker",
    auto_adjust=False,
    threads=True,
    progress=True
)

# -----------------------------
# 3️⃣ Convert MultiIndex → long format
# -----------------------------
df_long = (
    df_prices
    .stack(level=0)
    .reset_index()
    .rename(columns={"level_1": "Ticker"})
)

# Rename Date column if needed
df_long.rename(columns={"Date": "Trade_Date"}, inplace=True)

# -----------------------------
# 4️⃣ Join sector information
# -----------------------------
df_final = df_long.merge(
    df_sp500_output[["Ticker Symbol", "Sector"]],
    left_on="Ticker",
    right_on="Ticker Symbol",
    how="left"
).drop(columns=["Ticker Symbol"])

# -----------------------------
# 5️⃣ Add metadata (optional but nice)
# -----------------------------
df_final["Data_Source"] = "Yahoo Finance (yfinance)"
df_final["Extracted_At"] = datetime.now()

# -----------------------------
# 6️⃣ Reorder columns (Power BI friendly)
# -----------------------------
df_final = df_final[
    [
        "Trade_Date",
        "Ticker",
        "Sector",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
        "Data_Source",
        "Extracted_At",
    ]
]

# -----------------------------
# 7️⃣ Save to CSV
# -----------------------------
output_file = "sp500_prices_daily_10y.csv"
df_final.to_csv(output_file, index=False)

print(f"Saved {len(df_final):,} rows to {output_file}")


Number of tickers: 503


[**********************52%                       ]  260 of 503 completed$BF.B: possibly delisted; no price data found  (period=10y)
[**********************68%********               ]  343 of 503 completed$BRK.B: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")
[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: possibly delisted; no price data found  (period=10y)
['BRK.B']: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")
C:\Users\billg\AppData\Local\Temp\ipykernel_21108\220500920.py:30: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(level=0)


Saved 1,223,648 rows to sp500_prices_daily_10y.csv
